In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
import math

import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, BatchNormalization
from keras.callbacks import EarlyStopping

import datetime as dt

from vrae.vrae import VRAE
from torch.utils.data import TensorDataset
import torch

Using TensorFlow backend.
C:\tools\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\tools\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\tools\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\tools\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Pas

In [2]:
path = "C:/Studia/Magisterka/Faints-Prediction/"
BP_filename = "BP.csv"
HR_filename = "HR.csv"

train_indices =  [str(indx[0]) for indx in pd.read_csv(path + "DATA/training_set.txt").values.tolist()]
test_indices =  [str(indx[0]) for indx in pd.read_csv(path + "DATA/test_set.txt").values.tolist()]
validation_indices =  [str(indx[0]) for indx in pd.read_csv(path + "DATA/validation_set.txt").values.tolist()]
all_indices = train_indices + test_indices + validation_indices

In [3]:
def shift(xs, n):
    if n >= 0:
        return np.concatenate((np.full(n, np.nan), xs[:-n]))
    else:
        return np.concatenate((xs[-n:], np.full(-n, np.nan)))

    
def series_to_supervised(share_prices, timestamps, input_time_steps, dropnan=True):
    share_prices_df = pd.DataFrame(share_prices)
    timestamps_df = pd.DataFrame(timestamps)
    share_prices_timeseries = list()
    timestamps_timeseries = list()
    
    for i in range(input_time_steps-1, -1, -1):
        share_prices_timeseries.append(share_prices_df.shift(i))
        timestamps_timeseries.append(timestamps_df.shift(i))
    
    aggregated_share_prices = pd.concat(share_prices_timeseries, axis=1)
    aggregated_timestamps = pd.concat(timestamps_timeseries, axis=1)
    
    if dropnan:
        aggregated_share_prices.dropna(inplace=True)
        aggregated_timestamps.dropna(inplace=True)

    aggregated_timestamps = aggregated_timestamps.values
    aggregated_share_prices = aggregated_share_prices.values
    
    not_overlapping_indexes = range(0, 
                                    len(aggregated_share_prices), 
                                    input_time_steps)
    
    aggregated_timestamps = aggregated_timestamps[not_overlapping_indexes]
    aggregated_share_prices = aggregated_share_prices[not_overlapping_indexes]
    return aggregated_share_prices, aggregated_timestamps
 

def split(BP_data, HR_data, col, time_steps):
    X = []
    y = []

    BP_supervised, HR_supervised = series_to_supervised(BP_data[col], HR_data[col], time_steps)
    for BP_interval, HR_interval in zip(BP_supervised, HR_supervised):
        BP_HR_interval = []
        for BP_time_step, HR_time_step in zip(BP_interval, HR_interval):
            BP_HR_interval.append([BP_time_step, HR_time_step])
        X.append(BP_HR_interval)
        if labels[col] == 'Synkope': label = [0., 1.] 
        else: label = [1., 0.]
        y.append(label)
    return np.array(X), np.array(y)


def split_df(BP_data, HR_data, time_steps):
    X = np.array([])
    y = np.array([])
    for col in BP_data:
        X_single, y_single = split(BP_data, HR_data, col, time_steps)
        if X.size == 0:
            X = X_single
            y = y_single
        else:
            X = np.concatenate((X, X_single))
            y = np.concatenate((y, y_single))
    return np.array(X), np.array(y)

In [4]:
BP_data = pd.read_csv(path + "DATA/" + BP_filename, low_memory=False)[all_indices]
HR_data = pd.read_csv(path + "DATA/" + HR_filename, low_memory=False)[all_indices]
labels = {col: BP_data[col].iloc[0] for col in BP_data}

BP_max_value = BP_data.iloc[1:].astype(np.float32).max().max()
BP_min_value = BP_data.iloc[1:].astype(np.float32).min().min()
HR_max_value = HR_data.iloc[1:].astype(np.float32).max().max()
HR_min_value = HR_data.iloc[1:].astype(np.float32).min().min()

BP_scaler = MinMaxScaler().fit(np.array([BP_min_value, BP_max_value]).reshape(-1,1))
HR_scaler = MinMaxScaler().fit(np.array([HR_min_value, HR_max_value]).reshape(-1,1))

BP_data_scaled = BP_data.iloc[1:].astype(np.float32).copy()
BP_data_scaled[all_indices] = BP_scaler.transform(BP_data_scaled[all_indices])

HR_data_scaled = HR_data.iloc[1:].astype(np.float32).copy()
HR_data_scaled[all_indices] = HR_scaler.transform(HR_data_scaled[all_indices])

In [5]:

def downsample_to(n, df, mean_window):
    new_df = pd.DataFrame()
    for col in df:
        data = df[col][1:]
#         data_temp = data.count()
#         div = int(data.count() / n) - 1 # -1 bo jakies problemy sa ze avr_data po usrednieniu ma dlugosc < 200 i sa ujemne excessPoints
        avr_data = data.rolling(mean_window).mean()
#         avr_data = [_ for _ in list(avr_data) if np.isnan(_) != True]
#         temp = len(avr_data)
#         excess_points = len(avr_data) - n
#         avr_data = avr_data[excess_points:]
        new_df[col] = pd.Series(avr_data)
#         print(f"TS {col} processing. Div: {div}. Excess points: {excess_points}. Dl: {temp}. data len: {data_temp}")
    return new_df

BP_data_scaled = downsample_to(200, BP_data_scaled, 50)
HR_data_scaled = downsample_to(200, HR_data_scaled, 50)

In [9]:
BP_data_scaled['187'][51]

0.35329264402389526

In [6]:
# part_to_drop = 0.4
# BP_data_scaled_trimmed = BP_data_scaled.copy()
# HR_data_scaled_trimmed = HR_data_scaled.copy()
# for col in BP_data_scaled_trimmed:
#     n_rows_to_drop = int(part_to_drop * np.count_nonzero(~np.isnan(BP_data_scaled_trimmed[col])))

#     BP_data_scaled_trimmed[col] = shift(BP_data_scaled_trimmed[col], -n_rows_to_drop)
#     HR_data_scaled_trimmed[col] = shift(HR_data_scaled_trimmed[col], -n_rows_to_drop)

In [10]:
#32 -> 8 -> 2
#64 -> 16 -> 2
#128 -> 32 -> 2
#wybor parametrow

# time_steps = [32, 64, 128]
time_steps = [64, 32] 
hidden_sizes = [16, 8]
# r_layers = [0, 2, 3, 4]
# r_layers = [2]
# LSTM_cells = [1, 16, 64, 256]
# LSTM_cells = [64]
# neurons = [5, 20, 50]
# neurons = [5]
# epochs = [1, 2, 3, 4, 5]
epochs = [10, 40, 80]
# batch_size = [32, 64, 128]
batch_sizes = [16, 32, 64]
# dropout_rates = [0.2, 0.4, 0.0]
# clips = [True, False]

hidden_size = 60 # 30? 60?
hidden_layer_depth = 1 #2?
latent_length = 2 #10? 20?
batch_size = 32
learning_rate = 0.0005
n_epochs = 40 #20? 80?
dropout_rate = 0.2
optimizer = 'Adam'  # options: ADAM, SGD
cuda = False  # options: True, False
print_every = 30
clip = True  # options: True, False
max_grad_norm = 5
loss = 'MSELoss'  # options: SmoothL1Loss, MSELoss
block = 'LSTM'  # options: LSTM, GRU

In [11]:
def choose_color(label):
    if label[0] == 1.0:
        return 'r'
    elif label[0] == 0.0:
        return 'b'
    else:
        return 'm'
    
def plot_latent_space(X, Y, typ, directory):
    latent_space = vrae.transform(TensorDataset(torch.from_numpy(X)))
    
    lsl = latent_space.shape[0]

    X_BP_means = scale([np.mean(x[:,0]) for x in X], (0,100))
    X_HR_means = scale([np.mean(x[:,1]) for x in X], (0,100))
    label_colors = [choose_color(y) for y in Y]

    plt.figure(figsize=(12,8))
    plt.scatter(latent_space[:, 0], latent_space[:, 1], c=X_HR_means[:lsl], s=5, cmap='gist_rainbow')
    plt.savefig(f"{directory}/{typ}_HR_MEANS_LS.png")

    plt.figure(figsize=(12,8))
    plt.scatter(latent_space[:, 0], latent_space[:, 1], c=X_BP_means[:lsl], s=5, cmap='gist_rainbow')
    plt.savefig(f"{directory}/{typ}_BP_MEANS_LS.png")

    plt.figure(figsize=(12,8))
    plt.scatter(latent_space[:, 0], latent_space[:, 1], c=label_colors[:lsl], s=5)
    plt.savefig(f"{directory}/{typ}_LABELS_LS.png")
    plt.close('all')
    
def scale(x, out_range=(-1, 1), axis=None):
    domain = np.min(x, axis), np.max(x, axis)
    y = (x - (domain[1] + domain[0]) / 2) / (domain[1] - domain[0])
    return y * (out_range[1] - out_range[0]) + (out_range[1] + out_range[0]) / 2
            

In [13]:
plt.ioff()
for ts, h in zip(time_steps, hidden_sizes):
    X_train, y_train = split_df(BP_data_scaled[train_indices], HR_data_scaled[train_indices], ts)
    X_test, y_test = split_df(BP_data_scaled[test_indices], HR_data_scaled[test_indices], ts)
    train_dataset = TensorDataset(torch.from_numpy(X_train))
    test_dataset = TensorDataset(torch.from_numpy(X_test))
    sequence_length = X_train.shape[1]
    number_of_features = X_train.shape[2]
    for e in epochs:
        for bs in batch_sizes:            
            vrae = VRAE(sequence_length=sequence_length,
            number_of_features=number_of_features,
            hidden_size=h,
            hidden_layer_depth=hidden_layer_depth,
            latent_length=latent_length,
            batch_size=bs,
            learning_rate=learning_rate,
            n_epochs=e,
            dropout_rate=dropout_rate,
            optimizer=optimizer,
            cuda=cuda,
            print_every=print_every,
            clip=clip,
            max_grad_norm=max_grad_norm,
            loss=loss,
            block=block,
            dload="C:/Studia/Magisterka/Faints-Prediction/VAE")
            
            epoch_loss, recon_loss, kl_loss = vrae.fit(train_dataset)
            epoch_loss = round(epoch_loss, 2)
            recon_loss = round(recon_loss, 2)
            kl_loss = round(kl_loss, 2)
            directory = "VRAE_results_LS2_smoothed_nontrimmed/" + f"LOSS{epoch_loss}_RECONLOSS{recon_loss}_KLLOSS{kl_loss}_LATENT{latent_length}_TS{ts}_BS{bs}_E{e}_HIDDEN{hidden_layer_depth}({h})"
            os.mkdir(directory)
            vrae.save(f'{directory}/vrae.pth')
            plot_latent_space(X_train, y_train, 'TRAIN', directory)
            plot_latent_space(X_test, y_test, 'TEST', directory)
            del vrae

Epoch: 0
Batch 30, loss = 386.2720, recon_loss = 386.2020, kl_loss = 0.0700
Batch 60, loss = 204.8918, recon_loss = 204.6913, kl_loss = 0.2006
Batch 90, loss = 111.1813, recon_loss = 110.3297, kl_loss = 0.8516
Batch 120, loss = 23.0821, recon_loss = 22.1569, kl_loss = 0.9252
Batch 150, loss = 27.9236, recon_loss = 27.2759, kl_loss = 0.6477
Batch 180, loss = 24.7002, recon_loss = 24.0774, kl_loss = 0.6228
Batch 210, loss = 29.5209, recon_loss = 28.9779, kl_loss = 0.5430
Batch 240, loss = 23.0305, recon_loss = 22.5459, kl_loss = 0.4846
Batch 270, loss = 18.5034, recon_loss = 18.0435, kl_loss = 0.4598
Batch 300, loss = 15.0075, recon_loss = 14.3827, kl_loss = 0.6248
Batch 330, loss = 21.7591, recon_loss = 21.2302, kl_loss = 0.5288
Batch 360, loss = 26.8091, recon_loss = 26.3560, kl_loss = 0.4531
Batch 390, loss = 25.3493, recon_loss = 24.9438, kl_loss = 0.4056
Batch 420, loss = 27.9421, recon_loss = 27.2076, kl_loss = 0.7345
Batch 450, loss = 27.2324, recon_loss = 26.5308, kl_loss = 0.701

Batch 120, loss = 10.9030, recon_loss = 9.9369, kl_loss = 0.9661
Batch 150, loss = 10.3833, recon_loss = 9.3928, kl_loss = 0.9904
Batch 180, loss = 15.2105, recon_loss = 14.2221, kl_loss = 0.9884
Batch 210, loss = 10.1073, recon_loss = 9.1485, kl_loss = 0.9588
Batch 240, loss = 13.1697, recon_loss = 12.1299, kl_loss = 1.0398
Batch 270, loss = 13.7443, recon_loss = 12.8008, kl_loss = 0.9435
Batch 300, loss = 15.5630, recon_loss = 14.4648, kl_loss = 1.0982
Batch 330, loss = 11.5928, recon_loss = 10.5656, kl_loss = 1.0271
Batch 360, loss = 9.4141, recon_loss = 8.4753, kl_loss = 0.9388
Batch 390, loss = 21.8826, recon_loss = 20.8722, kl_loss = 1.0105
Batch 420, loss = 10.8146, recon_loss = 9.6236, kl_loss = 1.1910
Batch 450, loss = 13.4565, recon_loss = 12.4014, kl_loss = 1.0551
Batch 480, loss = 8.5544, recon_loss = 7.6140, kl_loss = 0.9404
Batch 510, loss = 9.0191, recon_loss = 8.0658, kl_loss = 0.9534
Average loss: 11.6368
Epoch: 8
Batch 30, loss = 20.1389, recon_loss = 19.1167, kl_loss

Average loss: 22.1643
Epoch: 9
Batch 30, loss = 25.0636, recon_loss = 23.7317, kl_loss = 1.3319
Batch 60, loss = 21.5332, recon_loss = 20.1707, kl_loss = 1.3626
Batch 90, loss = 32.7294, recon_loss = 31.3372, kl_loss = 1.3922
Batch 120, loss = 21.8754, recon_loss = 20.5646, kl_loss = 1.3109
Batch 150, loss = 20.6974, recon_loss = 19.4151, kl_loss = 1.2823
Batch 180, loss = 24.0999, recon_loss = 22.5972, kl_loss = 1.5027
Batch 210, loss = 10.0242, recon_loss = 8.6286, kl_loss = 1.3956
Batch 240, loss = 40.3593, recon_loss = 38.9447, kl_loss = 1.4146
Average loss: 21.8939
Epoch: 0
Batch 30, loss = 3058.7346, recon_loss = 3058.7197, kl_loss = 0.0149
Batch 60, loss = 2373.5874, recon_loss = 2373.5649, kl_loss = 0.0224
Batch 90, loss = 1483.9265, recon_loss = 1483.8093, kl_loss = 0.1172
Batch 120, loss = 417.7478, recon_loss = 417.0322, kl_loss = 0.7156
Average loss: 2188.6349
Epoch: 1
Batch 30, loss = 100.9731, recon_loss = 98.4669, kl_loss = 2.5062
Batch 60, loss = 111.1075, recon_loss = 

Batch 90, loss = 22.9474, recon_loss = 21.9446, kl_loss = 1.0027
Batch 120, loss = 26.4059, recon_loss = 25.4657, kl_loss = 0.9402
Batch 150, loss = 19.4964, recon_loss = 18.5004, kl_loss = 0.9960
Batch 180, loss = 11.9288, recon_loss = 10.8231, kl_loss = 1.1057
Batch 210, loss = 7.1220, recon_loss = 5.9166, kl_loss = 1.2053
Batch 240, loss = 12.7877, recon_loss = 11.7980, kl_loss = 0.9897
Batch 270, loss = 6.6418, recon_loss = 5.5448, kl_loss = 1.0970
Batch 300, loss = 10.2750, recon_loss = 9.1553, kl_loss = 1.1197
Batch 330, loss = 15.8349, recon_loss = 14.7616, kl_loss = 1.0733
Batch 360, loss = 9.6585, recon_loss = 8.7162, kl_loss = 0.9423
Batch 390, loss = 8.8478, recon_loss = 7.7518, kl_loss = 1.0960
Batch 420, loss = 9.0215, recon_loss = 7.9714, kl_loss = 1.0501
Batch 450, loss = 19.5319, recon_loss = 18.5844, kl_loss = 0.9475
Batch 480, loss = 14.1522, recon_loss = 12.8528, kl_loss = 1.2994
Batch 510, loss = 8.0010, recon_loss = 7.0106, kl_loss = 0.9903
Average loss: 11.6392
Ep

Batch 240, loss = 9.9569, recon_loss = 8.8717, kl_loss = 1.0851
Batch 270, loss = 12.5619, recon_loss = 11.3945, kl_loss = 1.1674
Batch 300, loss = 12.1786, recon_loss = 10.9988, kl_loss = 1.1798
Batch 330, loss = 9.1317, recon_loss = 7.8856, kl_loss = 1.2461
Batch 360, loss = 9.0016, recon_loss = 7.7129, kl_loss = 1.2888
Batch 390, loss = 18.3505, recon_loss = 17.1209, kl_loss = 1.2297
Batch 420, loss = 13.0407, recon_loss = 11.9121, kl_loss = 1.1286
Batch 450, loss = 13.0302, recon_loss = 11.9001, kl_loss = 1.1301
Batch 480, loss = 9.7370, recon_loss = 8.4197, kl_loss = 1.3173
Batch 510, loss = 10.3627, recon_loss = 9.2553, kl_loss = 1.1074
Average loss: 9.8599
Epoch: 12
Batch 30, loss = 11.8869, recon_loss = 10.7305, kl_loss = 1.1564
Batch 60, loss = 8.6040, recon_loss = 7.3023, kl_loss = 1.3017
Batch 90, loss = 6.8234, recon_loss = 5.4749, kl_loss = 1.3485
Batch 120, loss = 11.7279, recon_loss = 10.3140, kl_loss = 1.4139
Batch 150, loss = 6.7790, recon_loss = 5.6280, kl_loss = 1.15

...

Batch 150, loss = 18.9057, recon_loss = 17.6233, kl_loss = 1.2824
Batch 180, loss = 24.5946, recon_loss = 23.3526, kl_loss = 1.2420
Batch 210, loss = 23.2123, recon_loss = 21.9249, kl_loss = 1.2875
Batch 240, loss = 15.5741, recon_loss = 14.3225, kl_loss = 1.2516
Average loss: 21.4993
Epoch: 11
Batch 30, loss = 16.7348, recon_loss = 15.4924, kl_loss = 1.2424
Batch 60, loss = 21.4187, recon_loss = 20.1659, kl_loss = 1.2528
Batch 90, loss = 25.7193, recon_loss = 24.4927, kl_loss = 1.2266
Batch 120, loss = 25.2474, recon_loss = 24.0170, kl_loss = 1.2303
Batch 150, loss = 23.5685, recon_loss = 22.2955, kl_loss = 1.2731
Batch 180, loss = 20.2996, recon_loss = 19.0580, kl_loss = 1.2416
Batch 210, loss = 14.7618, recon_loss = 13.4949, kl_loss = 1.2669
Batch 240, loss = 21.8563, recon_loss = 20.5562, kl_loss = 1.3001
Average loss: 21.3798
Epoch: 12
Batch 30, loss = 21.1931, recon_loss = 19.9001, kl_loss = 1.2929
Batch 60, loss = 24.3715, recon_loss = 23.1220, kl_loss = 1.2495
Batch 90, loss = 

Batch 90, loss = 17.4969, recon_loss = 16.2380, kl_loss = 1.2589
Batch 120, loss = 14.3902, recon_loss = 13.1545, kl_loss = 1.2357
Batch 150, loss = 16.5762, recon_loss = 15.3476, kl_loss = 1.2285
Batch 180, loss = 17.8220, recon_loss = 16.6247, kl_loss = 1.1973
Batch 210, loss = 16.2786, recon_loss = 15.0666, kl_loss = 1.2121
Batch 240, loss = 15.0152, recon_loss = 13.7670, kl_loss = 1.2481
Average loss: 20.1221
Epoch: 26
Batch 30, loss = 21.6008, recon_loss = 20.2833, kl_loss = 1.3175
Batch 60, loss = 20.0212, recon_loss = 18.7799, kl_loss = 1.2413
Batch 90, loss = 14.6795, recon_loss = 13.4286, kl_loss = 1.2509
Batch 120, loss = 18.9614, recon_loss = 17.6945, kl_loss = 1.2669
Batch 150, loss = 16.2780, recon_loss = 15.0332, kl_loss = 1.2448
Batch 180, loss = 18.6588, recon_loss = 17.4250, kl_loss = 1.2338
Batch 210, loss = 25.1179, recon_loss = 23.8957, kl_loss = 1.2222
Batch 240, loss = 16.6139, recon_loss = 15.3902, kl_loss = 1.2237
Average loss: 19.3999
Epoch: 27
Batch 30, loss =

Batch 90, loss = 4.1699, recon_loss = 1.7734, kl_loss = 2.3965
Batch 120, loss = 3.7017, recon_loss = 1.2828, kl_loss = 2.4189
Batch 150, loss = 3.8227, recon_loss = 1.4179, kl_loss = 2.4048
Batch 180, loss = 3.7987, recon_loss = 1.3121, kl_loss = 2.4867
Batch 210, loss = 4.5001, recon_loss = 2.0438, kl_loss = 2.4563
Batch 240, loss = 3.5359, recon_loss = 1.1279, kl_loss = 2.4080
Average loss: 3.7561
Epoch: 41
Batch 30, loss = 3.0428, recon_loss = 0.7196, kl_loss = 2.3232
Batch 60, loss = 4.6175, recon_loss = 2.2264, kl_loss = 2.3911
Batch 90, loss = 3.2150, recon_loss = 0.8874, kl_loss = 2.3276
Batch 120, loss = 3.8005, recon_loss = 1.3661, kl_loss = 2.4344
Batch 150, loss = 3.3899, recon_loss = 1.0074, kl_loss = 2.3825
Batch 180, loss = 3.2503, recon_loss = 0.8785, kl_loss = 2.3718
Batch 210, loss = 3.5642, recon_loss = 1.1637, kl_loss = 2.4005
Batch 240, loss = 3.4239, recon_loss = 1.0551, kl_loss = 2.3688
Average loss: 3.6651
Epoch: 42
Batch 30, loss = 3.3697, recon_loss = 0.9804, 

Batch 150, loss = 3.0489, recon_loss = 0.7337, kl_loss = 2.3152
Batch 180, loss = 2.9571, recon_loss = 0.6993, kl_loss = 2.2578
Batch 210, loss = 2.9572, recon_loss = 0.6893, kl_loss = 2.2678
Batch 240, loss = 3.1431, recon_loss = 0.8548, kl_loss = 2.2883
Average loss: 3.3244
Epoch: 56
Batch 30, loss = 3.1969, recon_loss = 0.9354, kl_loss = 2.2615
Batch 60, loss = 3.1280, recon_loss = 0.8288, kl_loss = 2.2993
Batch 90, loss = 3.8206, recon_loss = 1.4276, kl_loss = 2.3931
Batch 120, loss = 3.0850, recon_loss = 0.7768, kl_loss = 2.3082
Batch 150, loss = 3.4405, recon_loss = 1.0379, kl_loss = 2.4026
Batch 180, loss = 3.1071, recon_loss = 0.8138, kl_loss = 2.2933
Batch 210, loss = 3.7574, recon_loss = 1.4595, kl_loss = 2.2979
Batch 240, loss = 2.9702, recon_loss = 0.6414, kl_loss = 2.3288
Average loss: 3.2901
Epoch: 57
Batch 30, loss = 3.0740, recon_loss = 0.8133, kl_loss = 2.2608
Batch 60, loss = 3.3833, recon_loss = 1.0890, kl_loss = 2.2943
Batch 90, loss = 3.4247, recon_loss = 1.0806, k

Batch 210, loss = 3.5487, recon_loss = 1.3026, kl_loss = 2.2461
Batch 240, loss = 2.8707, recon_loss = 0.6818, kl_loss = 2.1889
Average loss: 3.1780
Epoch: 71
Batch 30, loss = 3.0472, recon_loss = 0.8084, kl_loss = 2.2388
Batch 60, loss = 3.2865, recon_loss = 0.9912, kl_loss = 2.2954
Batch 90, loss = 3.3069, recon_loss = 1.0288, kl_loss = 2.2781
Batch 120, loss = 3.0904, recon_loss = 0.7856, kl_loss = 2.3048
Batch 150, loss = 3.0747, recon_loss = 0.8676, kl_loss = 2.2071
Batch 180, loss = 3.1772, recon_loss = 0.9536, kl_loss = 2.2235
Batch 210, loss = 3.1146, recon_loss = 0.7961, kl_loss = 2.3185
Batch 240, loss = 2.8189, recon_loss = 0.5646, kl_loss = 2.2544
Average loss: 3.1759
Epoch: 72
Batch 30, loss = 3.2198, recon_loss = 0.9576, kl_loss = 2.2623
Batch 60, loss = 2.9967, recon_loss = 0.7686, kl_loss = 2.2282
Batch 90, loss = 3.2356, recon_loss = 0.8739, kl_loss = 2.3617
Batch 120, loss = 3.0750, recon_loss = 0.8085, kl_loss = 2.2664
Batch 150, loss = 3.3930, recon_loss = 1.1153, k

In [ ]:
plt.plot()